## 1 Load WNUT 17 dataset

In [1]:
from pathlib import Path
import re


def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()  # 去除末尾（开头）的多余换行符
    raw_docs = re.split(r'\n\t?\n', raw_text)  # 以文档单位构成列表，用两个换行（可能中间有制表符）标识
    token_docs = []  # 元素为一个文档的所有token
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')  # 一行中包含token和tag（用制表符分隔）
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs


texts, tags = read_wnut('wnut17train.conll')

In [2]:
print(texts[0][10:17], tags[0][10:17], sep='\n')  # 第一个文档的部分token和tag

['for', 'two', 'weeks', '.', 'Empire', 'State', 'Building']
['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location']


## 2 Preprocess

In [3]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(
    texts, tags, test_size=.2)

In [4]:
# 映射
unique_tags = set(tag for doc in tags for tag in doc)  # 有多少种tag并按序排号
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()} # 标签用数字表示

In [5]:
# 对token编码/分词 便于输入到DistllBert
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True,
                            return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True,
                          return_offsets_mapping=True, padding=True, truncation=True)
# 文件包含input_ids，及编码，和offset_mapping，及子标签相对源标签的偏移
# 分词后还是[[文档1], [token1编码，token2编码], [token1子token编码，token1子token编码], ...]

In [6]:
# 原有的词语
print(train_texts[0])
# 将编码还原成token
tokens = tokenizer.convert_ids_to_tokens(train_encodings.input_ids[0])
print(tokens)
print(train_encodings.input_ids[0])

['next', 'few', 'weeks', 'are', 'going', 'to', 'be', 'hectic', 'between', 'traveling', 'and', 'knee', 'surgery', 'when', 'i', 'get', 'back']
['[CLS]', 'next', 'few', 'weeks', 'are', 'going', 'to', 'be', 'he', '##ctic', 'between', 'traveling', 'and', 'knee', 'surgery', 'when', 'i', 'get', 'back', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 1397, 1374, 2277, 1132, 1280, 1106, 1129, 1119, 11143, 120

In [7]:
import numpy as np
# 分词器将一个标记拆分为多个子标记，那么最终会出现标记和标签之间的不匹配


def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc]
              for doc in tags]  # 把tags转换映射id（未分词前）
    encoded_labels = []
    # offset_mapping指示子标记相对于从其拆分的原始标记的开始位置和结束位置。
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(
            len(doc_offset), dtype=int) * -100  # 长度以子标记数量为准
        arr_offset = np.array(doc_offset)

        # 将那些偏移首位为0，第二位不为零的子token，说明是原来token的第一个子token
        # 筛选下来数量就是分词器的token数，标记为对应的id，其他还是-100
        doc_enc_labels[(arr_offset[:, 0] == 0) & (
            arr_offset[:, 1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels


train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [11]:
train_labels[0]

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

## 3 Create a dataset

In [8]:
import torch


class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
                for key, val in self.encodings.items()} # items就是键、值数组
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# we don't want to pass this to the model, 仅保留input_ids，即分词编码
train_encodings.pop("offset_mapping")
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

In [9]:
train_dataset[0]

{'input_ids': tensor([  101,  1397,  1374,  2277,  1132,  1280,  1106,  1129,  1119, 11143,
          1206,  6934,  1105,  5656,  6059,  1165,   178,  1243,  1171,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0

## 4 Load model

In [10]:
from transformers import DistilBertForTokenClassification，TrainingArguments, Trainer
model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-cased', num_labels=len(unique_tags))

SyntaxError: invalid character in identifier (Temp/ipykernel_7416/1070711664.py, line 1)

In [ ]:
# 参数设置
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=WNUTDataset(train_encodings, train_labels)
    eval_dataset=tokenized_wnut["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)